<h1>Import all the needed libraries </h1>

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
# from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from pandas import json_normalize

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

import requests
!pip install lxml
import  lxml.html as lh
print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


<h1> Convert the table from wiki page to dataframe </h1>


In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)
doc = lh.fromstring(page.content)
tr_elements = doc.xpath('//tr')
#tr_head = doc.xpath('//th')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    name=name.rstrip('\n')
    col.append((name,[]))
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
   
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content()
        data = data.rstrip('\n')
        
       # print(data)
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)       
df = df.drop([180], axis=0) #remove unnecessary imported rows
df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


<h1>Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned</h1>

In [3]:
df['Neighborhood'] = df['Neighborhood'].str.replace(' /',', ') #replacing / with ,
df.rename(columns={"Postal code": "PostalCode"}, inplace=True) #renaming column name
df1= df[~df.Borough.str.contains("Not assigned")] #assigning dataframe with borough that is assigned
df1.reset_index(drop=True, inplace=True) # resetting index



<h1>check if a cell has a borough but a Not assigned neighborhood</h1>

In [4]:
df1.loc[df1['Neighborhood'] == 'Not assigned']

,PostalCode,Borough,Neighborhood


In [ ]:
#since there is no unassigned neighborhood we don't need to assign it to borough cell

<h1> find shape of dataframe </h1>

In [5]:
df1.shape

(103, 3)

In [6]:
df1.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [75]:
df1.to_csv('Toronto.csv',index=False)#convert df to excel for further use